## Thingking1:什么是监督学习，无监督学习，半监督学习

1. 监督学习
- 学习的数据集既有特征变量也有目标变量
2. 非监督学习
- 学习的数据集只有特征变量，而没有目标变量
3. 半监督学习
- 半监督学习介于监督学习和无监督学习之间
- 通常半监督学习的任务与监督学习一致，即任务中包含有明确的目标（如分类），采用的数据包括有标签的数据，也包括无标签的数据
- 作用：只有少量的数据有Label,利用没有标签的数据来学习整个数据的潜在分布

## Thinking2:K-means中的K值如何选取？
- 轮廓系数法：平均轮廓系数越大，聚类效果越好。但是轮廓系数确定出的最优K值不一定是最优的，有时候还需要根据SSE去辅助选取
- 手肘法：随着聚类数K的增加，样本划分会更加精细，每个簇的聚合程度会逐渐提高，那么误差平方和SSE（误差平方和）自然会逐渐变小，并且，当K小于真实聚类数时，由于K的增加会大幅度增加每个族的聚合程度，故SSE的下降幅度会很大，而当K到达真实聚类数时，再增加K所得到的聚合程度回报会`迅速变小`，所以SSE的下降幅度会骤减，然后随着K值的继续增大而趋于平缓，也就是说SSE和K的关系图是一个手肘的形状，而这个肘部对应的K值就是数据的真实聚类数。

## Thinking3：随机森林采用了 bagging 集成学习，bagging指的是什么

- bagging算法的原理类似投票，将若干个弱分类器的分类结果进行投票选择，按照“少数服从多数的原则”产生最终的预测结果，从而组成一个强分类器，这就是随机森林 bagging 思想，实际上 bagging 的思想是机器学习加强版。 

## Thinking4:表征学习和半监督学习的区别是什么？ 

1. 表征学习（representation),也称之为特征学习（feature learning),目的是对复杂的原始数据化繁为简，把原始的无效信息剔除，把有效信息更有效地进行提炼，形成特征
2. 半监督学习：通常半监督学习的任务与监督学习一致，即任务中包含有明确的目标（如分类），采取的数据既包含有标签的数据，也包括无标签的数据，只有少量的数据有label,利用没有标签的数据来学习整个数据的潜在分布。
3. 表征学习针对的是整个数据集，而半监督学习针对的是没有标签的数据

## Action1：文本抄袭自动检测

In [84]:
from collections import defaultdict
import os 
import pickle 

import numpy as np 
import pandas as pd 
import jieba
import editdistance 

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans 
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
# 加载停用词
with open('./L3/Assignment/chinese_stopwords.txt','r', encoding = 'utf-8') as file:
    stopwords = [[i[:-1] for i in file.readlines()]]

In [20]:
# 数据加载
news = pd.read_csv('./L3/Assignment/sqlResult.csv', encoding = 'gb18030')

In [27]:
news = news.dropna(subset = ['content'])

In [29]:
# 分词
def split_text(text):
    text = text.replace(' ','')
    text = text.replace('\n','')
    text2 = jieba.cut(text.strip())
    result = ' '.join([w for w in text2 if w not in stopwords])
    
    return result 

In [41]:
if not os.path.exists('corpus.pkl'):
    # 如果没有的话就对所有的数据做一个分割并且保存一下
    corpus = list(map(split_text,[str(i) for i in news.content]))
    with open('corpus.pkl','wb') as file:
        pickle.dump(corpus, file)
else:
    with open('corpus.pkl','rb') as file:
        corpus = pickle.load(file)

In [47]:
# 计算 corpus 的 TF-IDF矩阵
countvectorizer = CountVectorizer(encoding = 'gbl18030',min_df = 0.015)
tfidftransformer = TfidfTransformer()
countvector = countvectorizer.fit_transform(corpus)
tfidf = tfidftransformer.fit_transform(countvector)

In [53]:
# 自己标记是否是自己的新闻
label = list(map(lambda source: 1 if '新华' in str(source) else 0, news.source))

In [57]:
x_train, x_test, y_train, y_test = train_test_split(tfidf.toarray(), label, test_size = 0.2, random_state = 1)

In [58]:
# 定义一个多项式的朴素贝叶斯的分类器
clf = MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB()

In [60]:
# 对全量的数据进行预测
y_pred = clf.predict(tfidf.toarray())
y_label = np.array(label)
compare_news_index = pd.DataFrame({'prediction':y_pred,'labels':y_label})
compare_news_index.head()

,prediction,labels
0,0,0
1,0,0
2,0,0
3,1,1
4,1,0


In [61]:
'''
在这里找出可能 copy 的候选人
具体的是预测的是新华社，但是实际上不是新华社的新闻
通过下面的语句拿出来 所有 copy 语句的下
'''
# 计算所有可疑文章的 index 
copy_news_index = compare_news_index[(compare_news_index['prediction'] == 1) & (compare_news_index['labels'] == 0)]
print('可疑的文章数：',len(copy_news_index))

可疑的文章数： 2699


In [62]:
# 统计新华社文章的 index 
xinhuashe_news_index = compare_news_index[(compare_news_index['labels'] == 1)].index

In [63]:
xinhuashe_news_index

Int64Index([    3,    17,    25,    27,    39,    51,    65,    95,   102,
              111,
            ...
            87044, 87045, 87046, 87047, 87048, 87049, 87050, 87051, 87052,
            87053],
           dtype='int64', length=78855)

In [65]:
# 将文章进行聚类
normalizer = Normalizer()
scaled_array = normalizer.fit_transform(tfidf.toarray())

In [67]:
if not os.path.exists('label.pkl'):
    kmeans = KMeans(n_clusters = 25, random_state = 1)
    k_labels = kmeans.fit_predict(scaled_array)
    with open('label.pkl','wb') as file:
        pickle.dump(k_labels, file)
    print('k_labels.shape = ',k_labels.shape)
else:
    with open('label.pkl','rb') as file:
        k_labels = pickle.load(file)

In [68]:
if not os.path.exists('id_class.pkl'):
    # 创建 id_class 
    id_class = {index:class_ for index,class_ in enumerate(k_labels)}
    with open('id_class.pkl','wb') as file:
        pickle.dump(id_class, file)
else:
    with open('id_class.pkl','rb') as file:
        id_class = pickle.load(file)

In [72]:
# 创建反向映射的 class_id 
if not os.path.exists('class_id.pkl'):
    class_id = defaultdict(set)
    for index, class_ in id_class.items():
        # 只统计新华社发布的 class_id 
        if index in xinhuashe_news_index.tolist():
            class_id[class_].add(index)
    with open('class_id.pkl','wb') as file:
        pickle.dump(class_id, file)
else:
    with open('class_id.pkl','rb') as file:
        class_id = pickle.load(file)

In [76]:
# 找相似的文本
def find_similar_text(cpindex, top = 10):
    '''
    先从对应的类里面找，再从新华社发布的文章中进行查找
    '''
    dist_dict = {i:cosine_similarity(tfidf[cpindex], tfidf[i]) for i in class_id[id_class[cpindex]]}
    
    return sorted(dist_dict.items(), key = lambda x:x[1][0], reverse = True)[:top]


In [77]:
# 假设 cpindex = 3352 
cpindex = 3352 
similar_list = find_similar_text(cpindex, top = 3)
similar_list

[(3134, array([[0.96783235]])),
 (63511, array([[0.94457252]])),
 (29441, array([[0.94207551]]))]

In [81]:
print('怀疑抄袭:\n',news.iloc[cpindex].content)
# 找一篇相似的原文
print('*' *80)
similar2 = similar_list[0][0]
print('相似原文\n',news.iloc[similar2].content)

怀疑抄袭:
 　　中国5月份56座城市新建商品住宅价格环比上涨，4月份为58座上涨。5月份15个一线和热点二线城市房地产市场基本稳定，5月份房地产调控政策效果继续显现。
　　统计局：15个一线和热点二线城市房价同比涨幅全部回落
　　国家统计局城市司高级统计师刘建伟解读5月份房价数据
　　5月份一二线城市房价平均涨幅继续回落
　　国家统计局今日发布了2017年5月份70个大中城市住宅销售价格统计数据。对此，国家统计局城市司高级统计师刘建伟进行了解读。
　　一、15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落、9个城市环比下降或持平
　　5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内。
　　二、70个大中城市中一二线城市房价同比涨幅持续回落
　　5月份，70个城市中新建商品住宅和二手住宅价格同比涨幅比上月回落的城市分别有29和18个。其中，一二线城市同比涨幅回落尤其明显。据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点。
　　三、70个大中城市中房价环比下降及涨幅回落城市个数均有所增加
　　5月份，70个城市中新建商品住宅价格环比下降的城市有9个，比上月增加1个；涨幅回落的城市有26个，比上月增加3个。二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个。

********************************************************************************
相似原文
 　　国家统计局19日发布数据，5月份，15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落，其中9个城市环比下降或持平。这9个价格环比下降或持平的城市为：北京、上海、南京、杭州、合肥、福州、郑州、深圳、成都。
　　“5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基

In [85]:
# 计算编辑距离
print('编辑距离：',editdistance.eval(corpus[cpindex], corpus[similar2]))

编辑距离： 394
